In [3]:
import requests
headers = {'User-Agent': 'Rishabh K Sharma/1.0 (Contact: sharmarishabhk@gmail.com)'}
response = requests.get('https://example.com', headers=headers)

response

<Response [200]>

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.tools.retriever import create_retriever_tool


In [5]:
loader = WebBaseLoader("https://www.myvi.in/blog/new-sim-card-rules")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

rules_tool = create_retriever_tool(retriever, 'New_connection_rules', 'Search for information about rules on buying a new sim card. for any question about new sim card rules use this tool')

In [6]:
loader = WebBaseLoader("https://www.myvi.in/blog/idea-fancy-number")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

fancy_number_tool = create_retriever_tool(retriever, 'fancy_number_guide', 'Search for information about guide on buying a new fancy number sim card. for any question about new fancy number sim card guide use this tool')

In [8]:
loader = WebBaseLoader("https://www.myvi.in/help-support/faqs/how-to-port-my-number-to-vi")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

port_number_tool = create_retriever_tool(retriever, 'port_number_guide', 'Search for information about guide on porting number to vi. for any question about mobile number portibility guide use this tool')

In [10]:
loader = WebBaseLoader("https://www.myvi.in/new-connection/buy-prepaid-sim-connection-online")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text"))

retriever = vectordb.as_retriever()

FAQ_number_tool = create_retriever_tool(retriever, 'faq', 'Search for information about guide on frequently asked questions. for any question about new sim card connection use this tool')

In [11]:
tools = [rules_tool, fancy_number_tool, port_number_tool, FAQ_number_tool]

In [12]:
tools

[Tool(name='New_connection_rules', description='Search for information about rules on buying a new sim card. for any question about new sim card rules use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10b880fe0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10c0b1a00>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10b882980>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10c0b1a00>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial

In [13]:
from langchain_ollama.chat_models import ChatOllama

llm = ChatOllama(model="Llama3.2:3B", temprature=0)

In [14]:
llm

ChatOllama(model='Llama3.2:3B')

In [15]:
from langchain_classic import hub

prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
from langchain_classic.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)


In [17]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [21]:
from langchain_classic.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)


In [22]:
agent_executor

AgentExecutor(verbose=False, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanM

In [23]:
agent_executor.invoke({"input": "how can i buy a new sim"})

{'input': 'how can i buy a new sim',
 'output': "To buy a new SIM card, follow these steps:\n\n1. **Register with the telecom operator**: All Point-of-Sale (PoS) agents, franchisees, and distributors must register with respective telecom operators.\n2. **Purchase from authorized dealers**: Buy a new SIM card from authorized dealers or PoS agents to avoid counterfeit cards.\n3. **Provide digital KYC documentation**: To purchase a new SIM card or apply for a fresh SIM for an existing number, provide digital KYC (Know Your Customer) documentation through Aadhaar QR code scanning.\n4. **Limit on SIMs per ID**: Individuals can have a maximum of 9 SIM cards on a single ID, and bulk SIM issuance is only allowed for businesses with proper documentation.\n5. **90-day cooling period**: A disconnected SIM number won't be reissued for 90 days to prevent immediate misuse.\n\nAdditionally, it's essential to:\n\n* Familiarize yourself with the new SIM card rules and regulations\n* Ensure you have a v